In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1. Load data
df = pd.read_excel("MSDs Risk - Data.xlsx", sheet_name="Sheet1")

# 2. Define predictor and outcome columns
predictor_cols = [
    "Age", "Gender", "Height (cm)", "Weight (kg)", "BMI", 
    "Medical History", "Smoking Habits", "Alcohol Consumption", 
    "Physical Exercise", "Job Tenure (years)", "Working Days/Week", 
    "Work Duration (hrs/day)", "Work Breaks (hrs/day)", 
    "Table Height (cm)", "Sitting Height (cm)"
]
outcome_vars = [
    "Neck Pain", "Shoulder Pain", "Upper Back Pain ", 
    "Lower Back Pain ", "Elbow Pain", "Wrist/Hand Pain", 
    "Hips/Thighs Pain", "Knees Pain", "Ankles/Feet Pain"
]

# 3. One-hot encode categorical variables
cat_cols = ["Gender", "Medical History", "Smoking Habits", 
            "Alcohol Consumption", "Physical Exercise"]
num_cols = [c for c in predictor_cols if c not in cat_cols]

encoder = OneHotEncoder(drop="first", sparse=False)
X_cat = encoder.fit_transform(df[cat_cols])
cat_feature_names = encoder.get_feature_names_out(cat_cols)
X_num = df[num_cols].to_numpy()
X_full = np.hstack([X_num, X_cat])
feature_names = num_cols + list(cat_feature_names)

# 4. Calculate VIF and filter features
X_df_no_const = pd.DataFrame(X_full, columns=feature_names)
vif_data = pd.DataFrame()
vif_data["feature"] = X_df_no_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_df_no_const.values, i) for i in range(X_df_no_const.shape[1])]
selected_features = vif_data[vif_data["VIF"] < 10]["feature"].tolist()

# 5. Build filtered design matrix with constant
X_filtered_df = sm.add_constant(X_df_no_const[selected_features])

# 6. Run logistic regression for each outcome and extract p-values
pval_df = pd.DataFrame(index=["const"] + selected_features, columns=outcome_vars)

for outcome in outcome_vars:
    y = df[outcome].map({"Yes": 1, "No": 0})
    try:
        model = sm.Logit(y, X_filtered_df)
        result = model.fit(disp=0)
        pval_df[outcome] = result.pvalues
    except Exception:
        pval_df[outcome] = np.nan  # Handle cases where model fails to converge

# 7. Save results
pval_df.to_excel("Logistic_Regression_P_Values_Filtered.xlsx")
print("P-values saved to 'Logistic_Regression_P_Values_Filtered.xlsx'")


C:\Users\user\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\user\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\user\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2383: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\user\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2441: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))
C:\Users\user\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals


P-values saved to 'Logistic_Regression_P_Values_Filtered.xlsx'
